In [ ]:
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')

## Review

**Review Question 0**

In [ ]:
united = Table.read_table('united.csv')

In [ ]:
united.show(6)

**Reminder:** We are treating this dataset as the population of all United Airlines flights that occurred over the two month period.

In [ ]:
population_parameter = np.average(united.column('Delay'))
population_parameter

**Review Question 1**

In [ ]:
births = Table.read_table('baby.csv')
births.show(6)

**Reminder:** We are treating this dataset as a random sample from a hosptial system.

In [ ]:
births.hist('Birth Weight')

## New material

### A bell shape tells us more about a distribution

In [ ]:
def chebyshev(num_SDs):
    # returns the least proportion of the data in +/- num_SDs
    z = num_SDs
    return 1 - 1/z**2  

In [ ]:
chebyshev(make_array(1,2,3))

**Task**: Empirically verify that for a bell shaped distribution,
- the proportion of values within 1, 2 and 3 SDs of the mean is 68, 95, and 99.75 percent, respectively.

In [ ]:
birth_weight_mean = np.average(births.column('Birth Weight'))
birth_weight_mean

In [ ]:
birth_weight_sd = np.std(births.column('Birth Weight'))
birth_weight_sd

### The Central Limit Theorem (CLT)

**Task**: Show that the empirical distribution of the sample mean approaches a bell shape as the sample size (for each statistic used to create the distribution) increases.

- Though the CLT is a statement about the probability distribution of the statistic, the empirical distribution will look roughly like the probability distribution when the repetition size is large. Here, we set it to 10,000.

In [ ]:
def one_sample_mean(sample_size):
    """ 
    Takes a sample from the population of flights 
    and computes its mean
    """
    sampled_flights = united.sample(sample_size)
    return np.mean(sampled_flights.column('Delay'))

In [ ]:
def ten_thousand_sample_means(sample_size):
    means = make_array()
    for i in np.arange(10000):
        mean = one_sample_mean(sample_size)
        means = np.append(means, mean)
    return means

In [ ]:
def plot_sample_mean_distribution(sample_size):
    sample_means = ten_thousand_sample_means(sample_size)
    
    Table().with_column('Mean of ' + str(sample_size) + ' flight delays', 
                        sample_means).hist(bins=20)

    print('Population Average:', population_parameter)

In [ ]:
plot_sample_mean_distribution(10)

In [ ]:
plot_sample_mean_distribution(30)

In [ ]:
plot_sample_mean_distribution(100)

**Discussion Question** [2 min]

After rolling 1,000,000 fair 6-sided dice, which of these histograms would you expect to have a bell shape? 
- You might expect there to be one (or more than one)!

**1.** The histogram of outcomes of these million rolls

**2.** The histogram that results from computing the average outcome of these million rolls

**3.** The histogram that results from:
 - splitting the outcomes into 1,000 groups of 1,000 (in the order they occurred)
 - and computing the average outcome of each group


In [ ]:
die = make_array(1,2,3,4,5,6)

In [ ]:
one_million_die_rolls = np.random.choice(die,size = 1000000)

In [ ]:
Table().with_columns('Die Spot', one_million_die_rolls).hist(bins = np.arange(1,8))